In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn import manifold
from torch.utils.data import DataLoader
device='cpu'
import os
parent_path='/home/baly/projects/linear_pgw/pu_learning/MNIST'
#parent_path='/home/baly/projects/linear_pgw/pu_learning'
os.chdir(parent_path)

In [2]:
# Define a Convolutional Autoencoder model
class ConvAutoencoder(nn.Module):
    def __init__(self, embedding_dim=8):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=0),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=0, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1),
              nn.ReLU(),
            nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=0, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=1, padding=0),
            nn.Sigmoid(),
        )
        self.embedding_encoder = nn.Sequential(
            nn.Linear(128 * 2 * 2, 512),
            nn.ReLU(),
            nn.Linear(512, embedding_dim)
            )
        self.embedding_decoder = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 128 * 2 * 2)
            )

    def forward(self, x):
        z = self.encoder(x)
        z = z.view(z.size(0), -1)  # Flatten        
        embedding = self.embedding_encoder(z)
        #embedding = embedding/torch.norm(embedding,p=2,dim=-1,keepdim=True)
        xhat = self.decoder(self.embedding_decoder(embedding).view(x.size(0), 128, 2, 2))
        return xhat, embedding

In [3]:
batch_size=256

In [4]:
transform = transforms.ToTensor()
mnist_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test_loader = DataLoader(mnist_data, batch_size=batch_size, shuffle=False)
# Load your trained autoencoder
autoencoder = ConvAutoencoder(embedding_dim=4).to(device)
autoencoder.load_state_dict(torch.load('AE_MNIST_0.pt'))  # Load your model
autoencoder.eval()

ConvAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (5): ReLU()
    (6): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): Re

In [25]:
autoencoder.eval()
test_encode, test_targets = [], []
for x_val, y_val in mnist_test_loader:
    #print(x_val.shape)
    x_val = x_val.to(device)
    xhat,zhat = autoencoder(x_val)
    # yhat = model.decoder(zhat)
    test_encode.append(zhat.detach().numpy())
    test_targets.append(y_val.detach().numpy())
X_list=np.vstack(test_encode)
label_list=np.concatenate(test_targets)
MNIST_data=(X_list,label_list)
torch.save(MNIST,'/home/baly/projects/linear_pgw/pu_learning/data/MNIST.pt')


print('Embeddings are calculated')


Embeddings are calculated


In [27]:
MNIST[0].shape

(60000, 4)

In [21]:
A=torch.vstack()

tensor([[ -2.0414,   0.8055, -48.8841,  16.4660],
        [-14.5873,  25.7937, -25.8517,  63.5655],
        [  8.2878,  22.7776,  39.0892,   4.5647],
        ...,
        [-54.7844, -29.8298, -16.5093,  30.3908],
        [ 12.1600, -24.6894, -34.8477,  25.3909],
        [ -8.7653, -20.8837, -67.9662,  12.9058]])

In [18]:
x_val

torch.Size([96, 1, 28, 28])

In [13]:
mnist_test_loader